<a href="https://colab.research.google.com/github/tthogho1/CompareImage/blob/main/BLIP2_to_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

# New Section

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!ls /content/drive/
!ls /content/drive/MyDrive/
!cd /content/drive/MyDrive/
!unzip /content/drive/MyDrive/images.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/1667479462.jpg   
  inflating: images/1667479485.jpg   
  inflating: images/1667479495.jpg   
  inflating: images/1667479506.jpg   
  inflating: images/1667479515.jpg   
  inflating: images/1667479528.jpg   
  inflating: images/1667479557.jpg   
  inflating: images/1667479573.jpg   
  inflating: images/1667479583.jpg   
  inflating: images/1667479603.jpg   
  inflating: images/1667479647.jpg   
  inflating: images/1667479661.jpg   
  inflating: images/1667479672.jpg   
  inflating: images/1667479684.jpg   
  inflating: images/1667479690.jpg   
  inflating: images/1667479696.jpg   
  inflating: images/1667479705.jpg   
  inflating: images/1667479716.jpg   
  inflating: images/1667479720.jpg   
  inflating: images/1667479726.jpg   
  inflating: images/1667479739.jpg   
  inflating: images/1667479754.jpg   
  inflating: images/1667479765.jpg   
  inflating: images/1667479776.jpg   
  inflating: images/1667479784.jpg   

In [18]:
!ls /content/drive
!ls -l images

Streaming output truncated to the last 5000 lines.
-rw-r--r-- 1 root root  2072 Mar 24 04:19 1648821141.jpg
-rw-r--r-- 1 root root  4848 Mar 24 03:31 1648821192.jpg
-rw-r--r-- 1 root root  3148 Mar 24 04:06 1648821195.jpg
-rw-r--r-- 1 root root  2007 Mar 23 03:18 1648821225.jpg
-rw-r--r-- 1 root root  2113 Mar 24 03:17 1648821237.jpg
-rw-r--r-- 1 root root  3309 Mar 24 03:44 1648821279.jpg
-rw-r--r-- 1 root root  4160 Mar 24 03:26 1648821459.jpg
-rw-r--r-- 1 root root  4026 Mar 24 04:16 1648821466.jpg
-rw-r--r-- 1 root root  3498 Mar 24 03:24 1648821703.jpg
-rw-r--r-- 1 root root  3647 Mar 24 03:55 1648821746.jpg
-rw-r--r-- 1 root root  4733 Mar 24 04:18 1648821785.jpg
-rw-r--r-- 1 root root  4025 Mar 24 03:38 1648821793.jpg
-rw-r--r-- 1 root root  5091 Mar 24 03:46 1648821806.jpg
-rw-r--r-- 1 root root  5358 Mar 24 03:51 1648821811.jpg
-rw-r--r-- 1 root root  4896 Mar 24 04:12 1648821818.jpg
-rw-r--r-- 1 root root  4767 Mar 24 04:07 1648821840.jpg
-rw-r--r-- 1 root root  3802 Mar 24 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
!pip install salesforce-lavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [20]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

In [ ]:
raw_image = Image.open('/content/images/1224018415.jpg').convert('RGB')
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model, vis_processors, _ = load_model_and_preprocess(
   name="blip2_t5", model_type="pretrain_flant5xxl", is_eval=True, device=device
)
vis_processors.keys()


In [24]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [25]:
result = model.generate({"image": image})
print(result)

['a person is standing on top of a mountain in the fog']


In [27]:
raw_image = Image.open('/content/images/1018690405.jpg').convert('RGB')

In [28]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
result = model.generate({"image": image})
print(result)

['an aerial view of a ski slope in the mountains']


In [30]:
import os

dir_path = "/content/images/"

files = os.listdir(dir_path)
print(files)

['1666680571.jpg', '1359088404.jpg', '1670882978.jpg', '1437608523.jpg', '1670262968.jpg', '1434025466.jpg', '1596829979.jpg', '1645632487.jpg', '1630491176.jpg', '1666639538.jpg', '1604841134.jpg', '1666269035.jpg', '1672584571.jpg', '1667478464.jpg', '1565047544.jpg', '1654624394.jpg', '1496113860.jpg', '1666641205.jpg', '1596826579.jpg', '1338727302.jpg', '1513335192.jpg', '1596826542.jpg', '1668086548.jpg', '1626164089.jpg', '1666262233.jpg', '1423756853.jpg', '1606333293.jpg', '1340470271.jpg', '1604838936.jpg', '1535450196.jpg', '1604847302.jpg', '1477298546.jpg', '1666679158.jpg', '1369218732.jpg', '1249040951.jpg', '1602151088.jpg', '1622404867.jpg', '1575920637.jpg', '1596965467.jpg', '1651071719.jpg', '1667242819.jpg', '1252636599.jpg', '1596883636.jpg', '1606333673.jpg', '1666269658.jpg', '1596889783.jpg', '1260447492.jpg', '1459339390.jpg', '1366369853.jpg', '1597141972.jpg', '1604859022.jpg', '1599358341.jpg', '1596828407.jpg', '1596889701.jpg', '1553677642.jpg', '16048499

In [31]:
for file in files:
    print(file)
    raw_image = Image.open(dir_path + file).convert('RGB')
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    result = model.generate({"image": image})
    print(result)

1666680571.jpg
['a traffic camera image of a highway with cars on it']
1359088404.jpg
['an aerial view of a road in the middle of a snow covered field']
1670882978.jpg
['a busy street with many motorcyclists and cars']
1437608523.jpg
['a view of the city from above on a cloudy day']
1670262968.jpg
['a view from the top of a snowy mountain']
1434025466.jpg
['a webcam image of a road with snow on the ground']
1596829979.jpg
['a truck driving down a snowy road with snow on the ground']
1645632487.jpg
['an aerial view of a city with snow on the ground']
1630491176.jpg
['a webcam image of an open field with mountains in the background']
1666639538.jpg
['a car is driving down the road in the middle of an intersection']
1604841134.jpg
['traffic on a highway with cars driving on it']
1666269035.jpg
['an aerial view of cars at an intersection']
1672584571.jpg
['a car is driving down the highway on the side of the road']
1667478464.jpg
['this is an aerial view of a wooded area']
1565047544.jpg
[

KeyboardInterrupt: ignored